# Importing libraries

In [1]:
import src.extraction as ext
import src.vis as vis
import src.analysis as ana
import src.foursquare_api as frsq

In [2]:
from pymongo import MongoClient
import numpy as np
import pandas as pd
import time

import os
import requests
import json
from dotenv import load_dotenv

import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
import geopandas as gpd



import statistics as s

import haversine as hs
load_dotenv()

True

# Filtering and extracting the data

In [3]:
df = ext.filtering_companies()


In [4]:
df2 = ext.cleaning_companies_df(df)

In [5]:
df3_topcompanies = df2.loc[df2["city"].isin(['San Francisco', 'New York', 'London'])].sort_values("total_money_raised", ascending=False).groupby(["city"]).first().reset_index()
df3_topcompanies

,city,name,total_money_raised,latitude,longitude
0,London,OANDA,117.0,51.519204,-0.162610
1,New York,AOL,1000.0,40.731132,-73.991931
2,San Francisco,Xero,244.0,37.779507,-122.390710


In [6]:
airports = pd.read_csv("../Project_3_Company_Headquarters/dataframes/airports.csv")

airports_london = ext.creating_airports_df(airports, "London")
airports_newyork = ext.creating_airports_df(airports, "New York")
airports_sanfrancisco = ext.creating_airports_df(airports, "San Francisco")

In [7]:
air = {"London" : airports_london,
      "New York": airports_newyork,
      "San Francisco": airports_sanfrancisco}

ext.distances_airports((df3_topcompanies["latitude"].iloc[1], df3_topcompanies["longitude"].iloc[1]), airports_newyork)
ext.distances_airports((df3_topcompanies["latitude"].iloc[2], df3_topcompanies["longitude"].iloc[2]), airports_sanfrancisco)
ext.distances_airports((df3_topcompanies["latitude"].iloc[0], df3_topcompanies["longitude"].iloc[0]), airports_london)

# Accessing the API

In [8]:
token_fsq = os.getenv("key")


In [ ]:
bars = frsq.making_requests("Bar", df3_topcompanies)
nightclubs = frsq.making_requests("Night Club", df3_topcompanies)
elementary_schools = frsq.making_requests("Elementary School", df3_topcompanies)
nurseries = frsq.making_requests("Nursery School", df3_topcompanies)

In [10]:
london = ana.concat_dfs([bars, nightclubs, nurseries, elementary_schools, air], "London")
new_york = ana.concat_dfs([bars, nightclubs, nurseries, elementary_schools, air], "New York")
san_francisco = ana.concat_dfs([bars, nightclubs, nurseries, elementary_schools, air], "San Francisco")

In [11]:
london_weight = ana.city_weights(london)
newyork_weight = ana.city_weights(new_york)
sanfrancisco_weight = ana.city_weights(san_francisco)

In [12]:
ana.city_score(london_weight, newyork_weight, sanfrancisco_weight)

"The best city for the HQ is ['New York']"

# Maps

In [ ]:
map_london, map_newyork, map_sanfrancisco = vis.empty_maps(df3_topcompanies)

In [14]:
vis.creating_maps(df3_topcompanies.loc[df3_topcompanies['city'] == 'London'], london, map_london)
map_london

In [15]:
vis.creating_maps(df3_topcompanies.loc[df3_topcompanies['city'] == 'New York'], new_york, map_newyork)
map_newyork

In [16]:
vis.creating_maps(df3_topcompanies.loc[df3_topcompanies['city'] == 'San Francisco'], san_francisco, map_sanfrancisco)
map_sanfrancisco